# **Colab Pro notebook from https://github.com/TheLastBen/fast-stable-diffusion** *Alternatives : [RunPod](https://www.runpod.io/console/gpu-browse?template=runpod-stable-unified)  | [Paperspace](https://console.paperspace.com/github/TheLastBen/PPS?machine=Free-GPU)*
## **[Support](https://ko-fi.com/thelastben)**

In [1]:
import os
from pathlib import Path
from PIL import Image

# do not run twice
os.chdir(Path(os.getcwd()).parent.parent)

# Create a Path object for the directory
os.getcwd()


'/Users/diehnmanuel/PycharmProjects/Diffusion-models'

# Model Download

In [ ]:
# run in command line one level above the repo
git clone https://github.com/huggingface/diffusers.git
git lfs install
git clone --no-checkout https://huggingface.co/runwayml/stable-diffusion-v1-5
cd stable-diffusion-v1-5 
git config core.sparsecheckout true
echo -e 'scheduler\ntext_encoder\ntokenizer\nunet\nvae\nmodel_index.json\n!*.safetensors\n!*.fp16.bin\n!*.non_ema.bin' > .git/info/sparse-checkout
git checkout

# Dreambooth

In [92]:
SESSION_NAME = "Manu"
WORKSPACE = Path('DreamBooth/output').resolve()
INSTANCE_NAME = SESSION_NAME
OUTPUT_DIR = str(WORKSPACE / "models" / SESSION_NAME)
SESSION_DIR = WORKSPACE / 'Sessions' / SESSION_NAME
INSTANCE_DIR = str(SESSION_DIR / 'instance_images')
IMAGES_FOLDER = (Path('DreamBooth/input/') / SESSION_NAME).resolve()

CONFIG_DIR_ACCELERATE = str(Path('DreamBooth/config/accelerate_config.yaml').resolve())
CONFIG_DIR_TRAINING = str(Path('DreamBooth/config/training_config.yaml').resolve())
SCRIPT_DIR_DREAMBOOTH = str(Path('diffusers/examples/dreambooth/train_dreambooth.py').resolve())

Model_Version = '1.5'
MODEL_NAME = str(Path("stable-diffusion-v1-5").resolve())

!mkdir -p "$INSTANCE_DIR"
# copy for images to your input folder now


In [93]:
CONFIG_DIR_TRAINING

'/Users/diehnmanuel/PycharmProjects/Diffusion-models/DreamBooth/config/training_config.yaml'

In [81]:
# check if your images have the desired size and copy them
crop_size = 512

for filename in Path(IMAGES_FOLDER).glob('*.png'):
    print(filename)
    new_path_with_file = Path(INSTANCE_DIR) / Path(filename).name
    file = Image.open(filename)
    width, height = file.size
    if file.size != (crop_size, crop_size):
        file = file.resize((crop_size, crop_size), Image.LANCZOS)
    file.save(new_path_with_file, format="PNG")


/Users/diehnmanuel/PycharmProjects/Diffusion-models/DreamBooth/input/Manu/picmd_1.png


# Training

In [70]:
from subprocess import getoutput
import time
import random


training_steps = 350
learning_rate = 1e-6

resolution = 512
prec="fp16" #no

enable_text_encoder_training = True

save_checkpoint_every_n_steps = True 
save_checkpoint_every = 500

Seed=random.randint(1, 999999)
instance_prompt = "picmd"


In [95]:
from torch import mode
import yaml
def create_configs(
        config_dir_accelerate,
        config_dir_training,
        model_dir, 
        instance_dir,
        output_dir,
        prompt,
        learning_rate, 
        seed, 
        resolution, 
        precision, 
        training_steps
    ):

    config_accelerate = {
        'compute_environment': 'LOCAL_MACHINE',
        'distributed_type': 'NO',
        'num_machines': 1,
        'num_processes': 1,
        'mixed_precision': precision,
    }

    config_training = {
        'gradient_accumulation_steps': 1,
        'gradient_checkpointing': True,
        'instance_data_dir': instance_dir,
        'instance_prompt': prompt,
        'learning_rate': learning_rate,
        'lr_scheduler': 'linear',
        'lr_warmup_steps': 0,
        'max_train_steps': training_steps,
        'output_dir': output_dir,
        'pretrained_model_name_or_path': model_dir,
        'resolution': resolution,
        'seed': seed,
        'train_batch_size': 1,
        'use_8bit_adam': True,
    }
    
    # Write the configuration to a YAML file
    with open(config_dir_accelerate, 'w') as file:
        yaml.safe_dump(config_accelerate, file, default_flow_style=False)   

    # Write the configuration to a YAML file
    with open(config_dir_training, 'w') as file:
        yaml.safe_dump(config_training, file, default_flow_style=False)

    return config_training


config_training = create_configs(
    config_dir_accelerate=CONFIG_DIR_ACCELERATE,
    config_dir_training=CONFIG_DIR_TRAINING,
    model_dir=MODEL_NAME,
    instance_dir=INSTANCE_DIR,
    output_dir=OUTPUT_DIR,
    prompt=instance_prompt,
    learning_rate=learning_rate,
    seed=Seed,
    resolution=resolution,
    precision=prec,
    training_steps=training_steps,
)

In [96]:
config_training

{'gradient_accumulation_steps': 1,
 'gradient_checkpointing': True,
 'instance_data_dir': '/Users/diehnmanuel/PycharmProjects/Diffusion-models/DreamBooth/output/Sessions/Manu/instance_images',
 'instance_prompt': 'picmd',
 'learning_rate': 1e-06,
 'lr_scheduler': 'linear',
 'lr_warmup_steps': 0,
 'max_train_steps': 350,
 'output_dir': '/Users/diehnmanuel/PycharmProjects/Diffusion-models/DreamBooth/output/models/Manu',
 'pretrained_model_name_or_path': '/Users/diehnmanuel/PycharmProjects/Diffusion-models/stable-diffusion-v1-5',
 'resolution': 512,
 'seed': 987305,
 'train_batch_size': 1,
 'use_8bit_adam': True}

In [99]:
config_training['gradient_accumulation_steps']

1

In [106]:
def launch_training(config_dir_accelerate, config_training, script_dir):

    command = (
        f"accelerate launch --config_file={config_dir_accelerate} {script_dir} "
        f"--gradient_accumulation_steps {str(config_training['gradient_accumulation_steps'])} "
        f"--gradient_checkpointing "
        f"--instance_data_dir {config_training['instance_data_dir']} "
        f"--instance_prompt \"{config_training['instance_prompt']}\" "
        f"--learning_rate {str(config_training['learning_rate'])} "
        f"--lr_scheduler {config_training['lr_scheduler']} "
        f"--lr_warmup_steps {str(config_training['lr_warmup_steps'])} "
        f"--max_train_steps {str(config_training['max_train_steps'])} "
        f"--output_dir {config_training['output_dir']} "
        f"--pretrained_model_name_or_path {config_training['pretrained_model_name_or_path']} "
        f"--resolution {str(config_training['resolution'])} "
        f"--seed {str(config_training['seed'])} "
        f"--train_batch_size {str(config_training['train_batch_size'])} "
        f"--use_8bit_adam"
    )

    get_ipython().system(command)
    
launch_training(
    config_dir_accelerate=CONFIG_DIR_ACCELERATE,
    config_training=config_training,
    script_dir=SCRIPT_DIR_DREAMBOOTH,
)

/Users/diehnmanuel/opt/anaconda3/envs/Diffusion/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/diehnmanuel/opt/anaconda3/envs/Diffusion/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/diehnmanuel/opt/anaconda3/envs/Diffusion/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/diehnmanuel/opt/anaconda3/envs/Diffusion/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register